In [ ]:
# Import dependencies

import collections
import csv
import numpy as np
import pandas as pd
import os
import scipy as sp

from datetime import datetime, timedelta
from sklearn.cluster import DBSCAN
from sklearn.neighbors import DistanceMetric

In [ ]:
# In this cell we define the function used to compute the sparse distance matrix

"""Return the index of the first element x in a collection such that pred(x) is True."""
def find(coll, pred):
    for i, x in enumerate(coll):
        if pred(x):
            return i
    raise ValueError

"""Drop the first n elements from the left end of deq."""
def dropn(deq, n):
    for _ in range(n):
        deq.popleft()

"""
Create a sparse distance matrix of distances between points in the data set. Currently this function hard
codes certain things that we might want to make into parameters, such as the distance metric and the format
of the input CSV. In order to work correctly, this function requires that the input data is sorted by
increasing timestamp values.

infile - path to input CSV file
outfile - path where the output CSV file will be saved
time_threshold - only record distances where the time difference is smaller than this timedelta
space_threshold - only record distances if the space distance is smaller than this number
"""
def write_sparse_distance_matrix(infile, outfile, time_threshold, space_threshold):
    
    # We can make these parameters if we need to
    dist = DistanceMetric.get_metric("haversine")
    date_format = "%m/%d/%Y %H:%M"
    
    def go(rr, wr):
        left_index = 0
        right_index = 0
        
        time_window = collections.deque()  # timestamp
        space_window = collections.deque() # [lat, long]
        
        for row in rr:
            lat, long, current_ts = row['lat'], row['lon'], row['date']
            current_ts = datetime.strptime(current_ts, date_format)
            
            try:
                number_to_drop = find(time_window, lambda ts: current_ts - ts < time_threshold)
                left_index += number_to_drop
                dropn(time_window, number_to_drop)
                dropn(space_window, number_to_drop)
            except ValueError:
                left_index += len(time_window)
                time_window.clear()
                space_window.clear()
            
            current_coords = [lat, long]
            
            if len(space_window) > 0:
                distances = dist.pairwise(space_window, [current_coords])
                for i, d in enumerate(np.nditer(distances)):
                    if d < space_threshold:
                        wr.writerow({
                                "x": left_index + i,
                                "y": right_index,
                                "distance": d
                            })
                        wr.writerow({
                                "x": right_index,
                                "y": left_index + i,
                                "distance": d
                            })
            
            right_index += 1
            time_window.append(current_ts)
            space_window.append(current_coords)
        
    with open(infile) as f, open(outfile, 'w') as outf:
        rr = csv.DictReader(f)
        wr = csv.DictWriter(outf, fieldnames=["x", "y", "distance"])
        wr.writeheader()
        go(rr, wr)

In [ ]:
# Set the input and output locations

working_directory = "/Users/joshtok/Code/geo-clustering"
infile = os.path.join(working_directory, "summer-travel-gps-full.csv")
outfile = os.path.join(working_directory, "sparse_distance_matrix.csv")

# Set the threshold parameters for the distance calculation
# The space threshold should be the same value used as a parameter in DBSCAN below

time_threshold = timedelta(minutes=60)
space_threshold = 10

In [ ]:
# Write the sparse matrix into a CSV file

write_sparse_distance_matrix(infile, outfile, time_threshold, space_threshold)

In [ ]:
# Load the sparse distance matrix and compute clusters using DBSCAN

df = pd.read_csv(outfile)
distances = sp.sparse.csr_matrix((df["distance"], (df["x"], df["y"])))

model = DBSCAN(metric="precomputed", eps=space_threshold)
fit = model.fit(distances)